In [9]:
import random
from tenacity import retry


@retry
def do_something_unreliable():
    num = random.randint(0, 10)

    if num >= 5:
        print(num)
        raise IOError(f"Retry {num}: something is wrong!")
    else:
        return "Awesome sauce!"


do_something_unreliable()

7
10
7
5


'Awesome sauce!'

# Basic Retry

In [13]:
@retry
def never_gonna_give_you_up():
    print("Retry forever ignoring Exceptions, don't wait between retries")
    raise Exception


never_gonna_give_you_up()

In [15]:
from tenacity import stop_after_attempt


@retry(stop=stop_after_attempt(7))
def stop_after_7_attempts():
    print("Stopping after 7 attempts")
    raise Exception


stop_after_7_attempts()


Stopping after 7 attempts
Stopping after 7 attempts
Stopping after 7 attempts
Stopping after 7 attempts
Stopping after 7 attempts
Stopping after 7 attempts
Stopping after 7 attempts


RetryError: RetryError[<Future at 0x1ee58908d30 state=finished raised Exception>]

In [21]:
import time
from tenacity import stop_after_delay

# `stop_after_delay` 用于表示超时时间，即运行超过设置 timeout 值后，不再 retry

@retry(stop=stop_after_delay(1))
def stop_after_10s():
    print("Stopping after 10 seconds")
    time.sleep(0.1)
    raise Exception


stop_after_10s()

Stopping after 10 seconds
Stopping after 10 seconds
Stopping after 10 seconds
Stopping after 10 seconds
Stopping after 10 seconds
Stopping after 10 seconds
Stopping after 10 seconds
Stopping after 10 seconds
Stopping after 10 seconds
Stopping after 10 seconds


RetryError: RetryError[<Future at 0x1ee5998cb20 state=finished raised Exception>]

In [24]:
@retry(stop=(stop_after_delay(5) | stop_after_attempt(10)))
def stop_after_10s_or_5_retries():
    print("Stopping after 10 seconds or 5 retries")
    time.sleep(1)
    raise Exception


stop_after_10s_or_5_retries()

Stopping after 10 seconds or 5 retries
Stopping after 10 seconds or 5 retries
Stopping after 10 seconds or 5 retries
Stopping after 10 seconds or 5 retries
Stopping after 10 seconds or 5 retries


RetryError: RetryError[<Future at 0x1ee588a8a90 state=finished raised Exception>]

# Waiting before retrying


In [25]:
from tenacity import wait_fixed


# wait_fixed 用于设置等待时长

@retry(wait=wait_fixed(2))
def wait_2s():
    print("Wait 2 second between retries")
    raise Exception


wait_2s()

Wait 2 second between retries
Wait 2 second between retries
Wait 2 second between retries
Wait 2 second between retries
Wait 2 second between retries
Wait 2 second between retries
Wait 2 second between retries
Wait 2 second between retries


KeyboardInterrupt: 

In [1]:
from tenacity import retry
from tenacity import wait_random, wait_exponential

# wait_random 用于设置随机等待时长
# wait_exponential 用于设置指数等等待时长


@retry(wait=wait_random(min=1, max=10))
def wait_random_1_to_2s():
    print("Randomly wait 1 to 2 seconds between retries")
    raise Exception


wait_random_1_to_2s()

NameError: name 'retry' is not defined

In [6]:
from tenacity import retry
from tenacity import wait_random, wait_exponential
from datetime import datetime


@retry(wait=wait_exponential(multiplier=1, min=4, max=10))
def wait_exponential_1():
    print(datetime.now())
    print("Wait 2^x * 1 second between each retry starting with 4 seconds, then up to 10 seconds, then 10 seconds afterwards")
    raise Exception


wait_exponential_1()

2022-07-15 12:13:27.494023
Wait 2^x * 1 second between each retry starting with 4 seconds, then up to 10 seconds, then 10 seconds afterwards
2022-07-15 12:13:31.495644
Wait 2^x * 1 second between each retry starting with 4 seconds, then up to 10 seconds, then 10 seconds afterwards
2022-07-15 12:13:35.498500
Wait 2^x * 1 second between each retry starting with 4 seconds, then up to 10 seconds, then 10 seconds afterwards
2022-07-15 12:13:39.502277
Wait 2^x * 1 second between each retry starting with 4 seconds, then up to 10 seconds, then 10 seconds afterwards
2022-07-15 12:13:47.503681
Wait 2^x * 1 second between each retry starting with 4 seconds, then up to 10 seconds, then 10 seconds afterwards
2022-07-15 12:13:57.506610
Wait 2^x * 1 second between each retry starting with 4 seconds, then up to 10 seconds, then 10 seconds afterwards
2022-07-15 12:14:07.509433
Wait 2^x * 1 second between each retry starting with 4 seconds, then up to 10 seconds, then 10 seconds afterwards
2022-07-15 12

KeyboardInterrupt: 

In [ ]:
@retry(wait=wait_fixed(3) + wait_random(0, 2))
def wait_fixed_jitter():
    print("Wait at least 3 seconds, and add up to 2 seconds of random delay")
    raise Exception

In [ ]:
from tenacity import wait_random_exponential

# wait_random_exponential 用于设置指数时间


@retry(wait=wait_random_exponential(multiplier=1, max=60))
def wait_exponential_jitter():
    print("Randomly wait up to 2^x * 1 seconds between each retry until the range reaches 60 seconds, then randomly up to 60 seconds afterwards")
    raise Exception

In [8]:
from tenacity import wait_chain, wait_fixed


@retry(wait=wait_chain(
    *[wait_fixed(3) for i in range(3)] +
     [wait_fixed(7) for i in range(2)] +
     [wait_fixed(9)]))
def wait_fixed_chained():
    print("Wait 3s for 3 attempts, 7s for the next 2 attempts and 9s for all attempts thereafter")
    raise Exception


wait_fixed_chained()

Wait 3s for 3 attempts, 7s for the next 2 attempts and 9s for all attempts thereafter
Wait 3s for 3 attempts, 7s for the next 2 attempts and 9s for all attempts thereafter
Wait 3s for 3 attempts, 7s for the next 2 attempts and 9s for all attempts thereafter
Wait 3s for 3 attempts, 7s for the next 2 attempts and 9s for all attempts thereafter
Wait 3s for 3 attempts, 7s for the next 2 attempts and 9s for all attempts thereafter


KeyboardInterrupt: 

# Whether to retry

In [3]:
import time
from tenacity import retry_if_exception_type
from tenacity.retry import retry_if_exception_type
from tenacity.retry import retry_if_not_exception_type
from tenacity import retry
from tenacity import wait_fixed
from tenacity import stop_after_attempt, stop_after_delay

In [5]:

class ClientError(Exception):
    """Some type of client error."""

@retry(
    retry=retry_if_exception_type(IOError),
    wait=wait_fixed(2),
    stop=(stop_after_delay(4) | stop_after_attempt(3))
)
def might_io_error():
    time.sleep(3)
    print("Retry forever with no wait if an IOError occurs, raise any other errors")
    raise IOError


might_io_error()

Retry forever with no wait if an IOError occurs, raise any other errors
Retry forever with no wait if an IOError occurs, raise any other errors


RetryError: RetryError[<Future at 0x17900995ca0 state=finished raised OSError>]

In [7]:
@retry(
    retry=retry_if_not_exception_type(ClientError),
    wait=wait_fixed(2),
    stop=stop_after_attempt(3)
)
def might_client_error():
    print("Retry forever with no wait if any error other than ClientError occurs. Immediately raise ClientError.")
    raise ClientError


might_client_error()


Retry forever with no wait if any error other than ClientError occurs. Immediately raise ClientError.


ClientError: 

In [11]:
from tenacity import retry_if_result


def is_none_p(value):
    """Return True if value is None"""
    return value is None

@retry(retry=retry_if_result(is_none_p(2)))
def might_return_none():
    time.sleep(0.2)
    print("Retry with no wait if return value is None")


might_return_none()

Retry with no wait if return value is None


TypeError: 'bool' object is not callable

In [12]:
def is_none_p(value):
    """Return True if value is None"""
    return value is None

@retry(retry=(retry_if_result(is_none_p) | retry_if_exception_type()))
def might_return_none():
    print("Retry forever ignoring Exceptions with no wait if return value is None")


might_return_none()

Retry forever ignoring Exceptions with no wait if return value is None
Retry forever ignoring Exceptions with no wait if return value is None
Retry forever ignoring Exceptions with no wait if return value is None
Retry forever ignoring Exceptions with no wait if return value is None
Retry forever ignoring Exceptions with no wait if return value is None
Retry forever ignoring Exceptions with no wait if return value is None
Retry forever ignoring Exceptions with no wait if return value is None
Retry forever ignoring Exceptions with no wait if return value is None
Retry forever ignoring Exceptions with no wait if return value is None
Retry forever ignoring Exceptions with no wait if return value is None
Retry forever ignoring Exceptions with no wait if return value is None
Retry forever ignoring Exceptions with no wait if return value is None
Retry forever ignoring Exceptions with no wait if return value is None
Retry forever ignoring Exceptions with no wait if return value is None
Retry 

KeyboardInterrupt: 

In [14]:
@retry
def do_something():
    result = something_else()
    if result == 23:
       raise TryAgain


do_something()

KeyboardInterrupt: 

# Error Handling


In [18]:
@retry(reraise=True, stop=stop_after_attempt(3))
def raise_my_exception():
    raise ClientError

try:
    raise_my_exception()
except ClientError:
    ...

# Before and After Retry, and Logging

In [23]:
import logging
from tenacity import before_log

logging.basicConfig(stream=sys.stderr, level=logging.DEBUG)

logger = logging.getLogger(__name__)


class MyException(Exception):
    def __init__(self, message):
        self.message = message

    def __str__(self):
        return f"{self.__class__.__name__}(message='{self.message}')"



@retry(stop=stop_after_attempt(3), before=before_log(logger, logging.DEBUG))
def raise_my_exception():
    raise MyException("Fail")


raise_my_exception()

DEBUG:__main__:Starting call to '__main__.raise_my_exception', this is the 1st time calling it.
DEBUG:__main__:Starting call to '__main__.raise_my_exception', this is the 2nd time calling it.
DEBUG:__main__:Starting call to '__main__.raise_my_exception', this is the 3rd time calling it.


RetryError: RetryError[<Future at 0x17903efa2e0 state=finished raised MyException>]

# Before and After Retry, and Logging

In [29]:
import logging
from tenacity import before_log, after_log


logging.basicConfig(stream=sys.stderr, level=logging.DEBUG)

logger = logging.getLogger("after_log")

@retry(stop=stop_after_attempt(3), after=after_log(logger, logging.DEBUG))
def raise_my_exception():
    raise MyException("Fail")


raise_my_exception()

DEBUG:after_log:Finished call to '__main__.raise_my_exception' after 0.000(s), this was the 1st time calling it.
DEBUG:after_log:Finished call to '__main__.raise_my_exception' after 0.016(s), this was the 2nd time calling it.
DEBUG:after_log:Finished call to '__main__.raise_my_exception' after 0.016(s), this was the 3rd time calling it.


RetryError: RetryError[<Future at 0x17903e029a0 state=finished raised MyException>]

In [30]:
import logging
from tenacity import before_log, after_log, before_sleep_log

logging.basicConfig(stream=sys.stderr, level=logging.DEBUG)

logger = logging.getLogger(__name__)

@retry(stop=stop_after_attempt(3),
       before_sleep=before_sleep_log(logger, logging.DEBUG))
def raise_my_exception():
    raise MyException("Fail")


raise_my_exception()

DEBUG:__main__:Retrying __main__.raise_my_exception in 0.0 seconds as it raised MyException: MyException(message='Fail').
DEBUG:__main__:Retrying __main__.raise_my_exception in 0.0 seconds as it raised MyException: MyException(message='Fail').


RetryError: RetryError[<Future at 0x17903da6d90 state=finished raised MyException>]

# Statistics

In [43]:
@retry(stop=stop_after_attempt(3))
def raise_my_exception():
    raise MyException("Fail")

try:
    raise_my_exception()
except Exception:
    ...

raise_my_exception.retry.statistics

{'start_time': 101836.765,
 'attempt_number': 3,
 'idle_for': 0,
 'delay_since_first_attempt': 0.0}

# Custom Callbacks

In [36]:
from tenacity import stop_after_attempt, retry_if_result


def return_last_value(retry_state):
    """return the result of the last call attempt"""
    print(retry_state)
    return retry_state.outcome.result()

def is_false(value):
    """Return True if value is False"""
    return value is False

# will return False after trying 3 times to get a different result
@retry(stop=stop_after_attempt(3),
       retry_error_callback=return_last_value,
       retry=retry_if_result(is_false))
def eventually_return_false():
    return False


eventually_return_false()

<RetryCallState 1619267875408: attempt #3; slept for 0.0; last result: returned False>


False

# Other Custom Callbacks

In [41]:
import logging

logging.basicConfig(stream=sys.stderr, level=logging.DEBUG)

logger = logging.getLogger(__name__)

def my_before_sleep(retry_state):
    if retry_state.attempt_number < 1:
        loglevel = logging.INFO
    else:
        loglevel = logging.WARNING

    logger.log(
        loglevel, 'Retrying %s: attempt %s ended with: %s',
        retry_state.fn, retry_state.attempt_number, retry_state.outcome)

@retry(stop=stop_after_attempt(3), before_sleep=my_before_sleep)
def raise_my_exception():
    raise MyException("Fail")

try:
    raise_my_exception()
except Exception:
    pass


# Changing Arguments at Run Time

In [48]:
@retry(stop=stop_after_attempt(3))
def raise_my_exception():
    print(123)
    raise MyException("Fail")

try:
    raise_my_exception.retry_with(stop=stop_after_attempt(4))()
except Exception:
    pass

raise_my_exception.retry.statistics

123
123
123
123


{}

In [ ]:
def never_good_enough(arg1):
    raise Exception('Invalid argument: {}'.format(arg1))

def try_never_good_enough(max_attempts=3):
    retryer = Retrying(stop=stop_after_attempt(max_attempts), reraise=True)
    retryer(never_good_enough, 'I really do try')

# Retrying code block

In [50]:
from tenacity import Retrying, RetryError, stop_after_attempt


try:
    for attempt in Retrying(stop=stop_after_attempt(3)):
        with attempt:
            print(123)
            raise Exception('My code is failing!')
except RetryError:
    pass

123
123
123


In [2]:
import asyncio
from tenacity import AsyncRetrying, RetryError, stop_after_attempt


async def function():
   try:
       async for attempt in AsyncRetrying(stop=stop_after_attempt(3)):
           with attempt:
               print(123)
               raise Exception('My code is failing!')
   except RetryError:
       pass


await function()

123
123
123


# Async and retry

In [ ]:
import time


@retry
async def my_async_function(loop):
    time.sleep(1)
    print(123)
    await loop.getaddrinfo('8.8.8.8', 53)


# await my_async_function(123)

CancelledError: 

In [9]:
await my_async_function(123)

123
123
123
123
123
123
123
123


CancelledError: 

In [ ]:
import time


@retry
@tornado.gen.coroutine
def my_async_function(http_client, url):
    yield http_client.fetch(url)

In [ ]:
@retry(sleep=trio.sleep)
async def my_async_function(loop):
    await asks.get('https://example.org')


await my_async_function()